# Plots for under-5 Prevalence of Malaria (DHS data)

## Preliminary steps

In [ ]:
rm(list = ls())

options(scipen=999)

# Global paths
Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
Sys.setenv(GDAL_DATA = "/opt/conda/share/gdal")

In [ ]:
# Paths
ROOT_PATH <- '~/workspace'
CONFIG_PATH <- file.path(ROOT_PATH, 'configuration')
CODE_PATH <- file.path(ROOT_PATH, 'code')
DATA_PATH <- file.path(ROOT_PATH, 'data')
DHS_DATA_PATH <- file.path(DATA_PATH, 'dhs', 'raw')
OUTPUT_DATA_PATH <- file.path(DATA_PATH, 'dhs', 'indicators', 'prevalence')
OUTPUT_PLOTS_PATH <- file.path(ROOT_PATH, 'pipelines', 'snt_dhs_indicators', 'reporting', 'outputs')

In [ ]:
# Load utils
source(file.path(CODE_PATH, "snt_utils.r"))

# List required pcks
required_packages <- c("sf", "glue", "data.table", "ggplot2", "stringi", "jsonlite", "httr", "reticulate")

# Execute function
install_and_load(required_packages)

Sys.setenv(RETICULATE_PYTHON = "/opt/conda/bin/python")
reticulate::py_config()$python
openhexa <- import("openhexa.sdk")

# Load SNT config
CONFIG_FILE_NAME <- "SNT_config.json"
config_json <- tryCatch({ fromJSON(file.path(CONFIG_PATH, CONFIG_FILE_NAME)) },
                        error = function(e) {
                          msg <- paste0("Error while loading configuration", conditionMessage(e))  
                          cat(msg)   
                          stop(msg) 
                        })

msg <- paste0("SNT configuration loaded from  : ", file.path(CONFIG_PATH, CONFIG_FILE_NAME)) 
log_msg(msg)

# Set config variables
COUNTRY_CODE <- config_json$SNT_CONFIG$COUNTRY_CODE

print(paste("Country code: ", COUNTRY_CODE))

## Geo data

In [ ]:
admin_level <- 'ADM1'
admin_id_col <- glue(admin_level, 'ID', .sep='_')
admin_name_col <- glue(admin_level, 'NAME', .sep='_')
admin_cols <- c(admin_id_col, admin_name_col)

In [ ]:
# Load spatial file from dataset

dhis2_dataset <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED

spatial_data_filename <- paste(COUNTRY_CODE, "shapes.geojson", sep = "_")
# spatial_data <- read_sf(file.path(DATA_PATH, 'dhis2', 'formatted', spatial_data_filename))
spatial_data <- get_latest_dataset_file_in_memory(dhis2_dataset, spatial_data_filename)
log_msg(glue("File {spatial_data_filename} successfully loaded from dataset version: {dhis2_dataset}"))

spatial_data <- st_as_sf(spatial_data)

# aggregate geometries by the admin columns
spatial_data <- aggregate_geometry(
  sf_data=spatial_data,
  admin_id_colname=admin_id_col,
  admin_name_colname=admin_name_col
)

# keep class
spatial_data <- st_as_sf(spatial_data)

if(COUNTRY_CODE == "COD"){
  spatial_data[[admin_name_col]] <- clean_admin_names(spatial_data[[admin_name_col]])
}

## Map

In [ ]:
data_source <- 'DHS'
indicator_u5prev <- 'PCT_U5_PREV_RDT'

In [ ]:
filename_without_extension <- glue("{COUNTRY_CODE}_{data_source}_{admin_level}_{toupper(indicator_u5prev)}")
u5_rdt_prevalence_table <- fread(file.path(OUTPUT_DATA_PATH, paste0(filename_without_extension, '.csv')))

plot_data =  merge(spatial_data, u5_rdt_prevalence_table, by = admin_cols, all = TRUE)

In [ ]:
lower_bound_col <- glue("{toupper(indicator_u5prev)}_CI_LOWER_BOUND")
upper_bound_col <- glue("{toupper(indicator_u5prev)}_CI_UPPER_BOUND")
sample_avg_col <- glue("{toupper(indicator_u5prev)}_SAMPLE_AVERAGE")

In [ ]:
u5_rdt_prevalence_plot <- make_dhs_map(
      plot_dt = plot_data,
      plot_colname = sample_avg_col,
      title_name = "Under-5 malaria prevalence (RDT, %)",
      legend_title = '%',
      scale_limits = c(0, 100)
    )

In [ ]:
plot_filename <- glue("{COUNTRY_CODE}_{data_source}_{admin_level}_{toupper(indicator_u5prev)}_plot.png")
ggsave(u5_rdt_prevalence_plot, file = file.path(OUTPUT_PLOTS_PATH, plot_filename), dpi = 500)

## Confidence interval plot

In [ ]:
ci_plot_title <- glue("{COUNTRY_CODE} {data_source} {toupper(indicator_u5prev)} CI")
ci_plot_xlab <- admin_name_col
ci_plot_ylab <- glue("% children with positive malaria RDT")
ci_plot_filename <- glue("{COUNTRY_CODE}_{data_source}_{admin_level}_{toupper(indicator_u5prev)}_CI_plot.png")
ci_u5_rdt_prevalence_plot <- make_ci_plot(
  df_to_plot=plot_data,
  admin_colname=admin_name_col,
  point_estimation_colname=sample_avg_col,
  ci_lower_colname=lower_bound_col,
  ci_upper_colname=upper_bound_col,
  title_name=ci_plot_title,
  x_title=ci_plot_xlab,
  y_title=ci_plot_ylab
)
# ci_u5_rdt_prevalence_plot
ggsave(filename=file.path(OUTPUT_PLOTS_PATH, ci_plot_filename), plot=ci_u5_rdt_prevalence_plot, width = 8, height = 6, dpi = 300)